In [1]:
import sys
import pandas as pd
import numpy as np
import tensorflow as tf
import math
import json
import os
from datetime import datetime
from keras.callbacks import History
history = History()

### Parameters to specify

In [2]:
#specify the number of bits:
#example: Q16.8 -> integer = 16, precision = 8, wordsize = 24, double = 48
PRECISION = 8
INTEGER = 8

#specify the files with paramaters, weights, test dataset and test dataset labels
use_case = "stream-video"
topo_str = "32x32x4"

dataset_folder = f"../datasets/nprint-raw/{use_case}/"
features_rankings = f"../datasets/nprint-raw/{use_case}/feature-importance.csv"
path_load_parameters = f"../tf-params-reports/nn-nprint-{use_case}-model-parameters-{topo_str}.json"
path_test = f'../datasets/nprint-raw/{use_case}/X_val.csv'            
path_label_test = f'../datasets/nprint-raw/{use_case}/y_val.csv'
path_load_model = f'../tf-models/nn-nprint-{use_case}-model-{topo_str}.keras'
title_for_analiser = f"../tf-csv-result-files/tf-out-class-{use_case}-{topo_str}-q{INTEGER}_{PRECISION}.csv"


newpath = f"../p4-vm/{use_case}-{topo_str}-q{PRECISION}-{INTEGER}"
topo_folder = f"../p4-vm/{use_case}-{topo_str}-q{PRECISION}-{INTEGER}/code/topology"
csv_input_folder = f"../p4-vm/{use_case}-{topo_str}-q{PRECISION}-{INTEGER}/code/csv-files-input"
csv_output_folder = f"../p4-vm/{use_case}-{topo_str}-q{PRECISION}-{INTEGER}/code/csv-files-output"
if not os.path.exists(newpath):
    os.makedirs(newpath)
if not os.path.exists(topo_folder):
    os.makedirs(topo_folder)
if not os.path.exists(csv_input_folder):
    os.makedirs(csv_input_folder)
if not os.path.exists(csv_output_folder):
    os.makedirs(csv_output_folder)

path_df_test_vm = f"../p4-vm/{use_case}-{topo_str}-q{PRECISION}-{INTEGER}/code/csv-files-input/df-test-{use_case}.csv"
path_target_test_vm = f"../p4-vm/{use_case}-{topo_str}-q{PRECISION}-{INTEGER}/code/csv-files-input/target-test-{use_case}.csv" 
p4_file_title = f"../p4-vm/{use_case}-{topo_str}-q{PRECISION}-{INTEGER}/code/ANN.p4"
tester_file_title = f"../p4-vm/{use_case}-{topo_str}-q{PRECISION}-{INTEGER}/code/NN-tester.py"

In [3]:
WORDSIZE = INTEGER + PRECISION
D_WORDSIZE = 2*WORDSIZE
SLACK = 8-(WORDSIZE%8)

n_bits = WORDSIZE
wordsize = WORDSIZE
precision = PRECISION
slack = SLACK

#specify the number of switches to distribute de neurons
number_of_input_switches = 1
number_of_hidden_switches = 1
number_of_output_switches = 1

### Load NN parameters and model

In [4]:
#Load file with NN topology
with open(path_load_parameters, "r") as f:
  rep = json.load(f)
print(rep)

#retrieve NN topology
number_of_attributes = rep["number_of_attributes"]
hidden_layer_nodes = rep["hidden_layer_nodes"]
output_layer_nodes = rep["output_layer_nodes"]
accuracy_test = rep["accuracy_test"]
max_number_of_neurons = max(number_of_attributes,hidden_layer_nodes,output_layer_nodes)

#select a sub-dataframe with only the top attrs rate by the AUTOGLUON feature importance algorithm
#load the file with the rankings
features_rankings_df = pd.read_csv(features_rankings)
features_rankings_df.rename(columns={'0': 'label'}, inplace=True)

#make a list with only the best features
feature_list = features_rankings_df['Unnamed: 0'].tolist()
#feature_list = ["pkt_1_ipv4_tl_5","pkt_1_udp_len_9","pkt_0_udp_len_12","pkt_2_ipv4_tl_6","pkt_4_udp_len_8","pkt_3_udp_len_7","pkt_2_ipv4_tl_7","pkt_1_udp_len_8","pkt_3_ipv4_tl_5","pkt_0_udp_len_15","pkt_3_udp_len_15","pkt_1_udp_len_10","pkt_3_udp_len_9","pkt_0_ipv4_tl_10","pkt_2_udp_len_11","pkt_2_ipv4_tl_10","pkt_4_udp_len_7","pkt_1_udp_len_13","pkt_0_ipv4_tl_12","pkt_1_ipv4_tl_9","pkt_0_ipv4_ttl_6","pkt_3_ipv4_tl_11","pkt_2_ipv4_ttl_2","pkt_4_udp_dport_0","pkt_0_ipv4_ttl_2","pkt_1_udp_dport_1","pkt_2_udp_len_12"]
feature_list = feature_list[:number_of_attributes]
#print(feature_list)

#read dataset and
#select a sub-dataframe with only the 32 top attrs rate by the AUTOGLUON feature importance algorithm
df_test = pd.read_csv(path_test)
label_test = pd.read_csv(path_label_test)
df_test = df_test[feature_list]
df_test.to_csv(path_df_test_vm, index=False)
#label_test.to_csv(path_target_test_vm, index=False)
#print(df_test)

{'datetime': '26-07-2024-16-09-54', 'number_of_attributes': 32, 'hidden_layer_nodes': 32, 'output_layer_nodes': 4, 'accuracy_test': 0.9536153078079224, 'batch_size': 512, 'epochs': 500}


In [5]:
#load model to get weights
model = tf.keras.models.load_model(path_load_model)
model.summary()

norm_layer_weights = model.layers[0].get_weights()[0] #weight
norm_layer_biases  = model.layers[0].get_weights()[1] #bias
hidden_layer_weights = model.layers[1].get_weights()[0] #weight
hidden_layer_biases  = model.layers[1].get_weights()[1] #bias
out_layer_weights = model.layers[2].get_weights()[0] #weight
out_layer_biases  = model.layers[2].get_weights()[1] #bias
print('\nInput -> Normalization Layer bias:\n',norm_layer_biases)
print('\nInput -> Normalization Layer weights:\n',norm_layer_weights)
print('\nNormalization -> Hidden Layer bias:\n',hidden_layer_biases)
print('\nNormalization -> Hidden Layer weights:\n',hidden_layer_weights)
print('\nHidden -> Output Layer bias:\n',out_layer_biases)
print('\nHidden -> Output Layer weights:\n',out_layer_weights,'\n')

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ normalization (Normalization)        │ (None, 32)                  │              65 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 32)                  │           1,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │             132 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,631 (14.19 KB)

 Trainable params: 1,188 (4.64 KB)

 Non-trainable params: 65 (264.00 B)

 Optimizer params: 2,378 (9.29 KB)


Input -> Normalization Layer bias:
 [3.2496046e-02 2.1035653e-01 2.6068407e-01 8.7901993e+01 2.6656783e-01
 2.4136128e-01 2.2219403e+02 1.4282243e+02 2.5592527e-01 2.3592572e-01
 2.5280026e-01 3.0108793e-02 4.6279755e+02 2.5586393e-01 2.3382136e-01
 2.3119856e-01 2.7180219e-01 4.3021112e+02 1.7713346e-01 2.4548876e-01
 2.1863680e-01 4.9189352e+02 2.3726189e-01 1.9919359e+01 2.2585610e-01
 2.5386244e-01 1.5285288e-01 2.1156773e-01 2.2041728e-01 5.5625623e+02
 2.4989355e-01 1.9132951e-01]

Input -> Normalization Layer weights:
 [0.96831894 0.30089435 0.3944217  0.9774101  0.42746702 0.32545096
 1.8325263  1.2299721  0.5115962  0.6342277  0.53372747 0.03107473
 3.3500943  0.62634534 0.6352888  0.6371078  0.50477487 3.3187194
 0.7738366  0.63362134 0.26936486 3.3010285  0.3628922  0.2995473
 0.28558436 0.45308474 0.15507048 0.7013794  0.31120205 3.817457
 0.48961648 0.25405487]

Normalization -> Hidden Layer bias:
 [ 0.43173155  0.04158311  0.2371507   0.04819361  0.24129122 -0.2554991
  

In [6]:
def num_to_c2(num, wordsize):
  if(num < 0):
    return 2**wordsize + num
  return num

### Generation of the Packet sender and receiver script

In [7]:
# print(df_test.columns)
# columns_str = "(neuron_id=0, "
# for i,x in enumerate(df_test.columns):
#   columns_str += f'data_{i+1}=tc["{x}"], '

# columns_str += "run_id=tc_id)"

# print(columns_str)



Index(['pkt_3_ipv4_src_23', 'pkt_0_ipv4_src_25', 'pkt_9_ipv4_dst_5',
       'pkt_2_rts', 'pkt_9_ipv4_dst_6', 'pkt_9_ipv4_dst_0', 'pkt_4_rts',
       'pkt_3_rts', 'pkt_7_ipv4_dst_27', 'pkt_6_tcp_opt_60',
       'pkt_6_ipv4_dst_4', 'pkt_0_ipv4_src_18', 'pkt_7_rts',
       'pkt_9_ipv4_src_28', 'pkt_4_tcp_opt_60', 'pkt_0_tcp_opt_60',
       'pkt_9_ipv4_dst_4', 'pkt_8_rts', 'pkt_4_ipv4_dst_1',
       'pkt_8_ipv4_src_28', 'pkt_9_ipv4_dst_3', 'pkt_5_rts',
       'pkt_7_ipv4_dst_11', 'pkt_1_rts', 'pkt_9_ipv4_src_21',
       'pkt_7_ipv4_dst_9', 'pkt_9_ipv4_dst_13', 'pkt_4_ipv4_dst_12',
       'pkt_7_ipv4_dst_0', 'pkt_6_rts', 'pkt_2_ipv4_dst_19',
       'pkt_3_ipv4_dst_7'],
      dtype='object')
(neuron_id=0, data_1=tc["pkt_3_ipv4_src_23"], data_2=tc["pkt_0_ipv4_src_25"], data_3=tc["pkt_9_ipv4_dst_5"], data_4=tc["pkt_2_rts"], data_5=tc["pkt_9_ipv4_dst_6"], data_6=tc["pkt_9_ipv4_dst_0"], data_7=tc["pkt_4_rts"], data_8=tc["pkt_3_rts"], data_9=tc["pkt_7_ipv4_dst_27"], data_10=tc["pkt_6_tcp_opt_60"]

In [8]:
tester = f'''
import sys
import socket
import struct
import re
import math
import json
import pandas as pd
import numpy as np
import threading
import queue
from scapy.all import *

# Creating the config dict in loco, maybe best would be to be in a file
cfg = {{
    "input_dataset_filename": "csv-files-input/df-test-{use_case}.csv",
    "output_csv_filename": "csv-files-output/p4-out-{use_case}-{topo_str}-q{INTEGER}_{PRECISION}.csv",
    "input_switches": [
        {{
            "name": "s1",
            "iface": "s1-eth1",
            "id": "s1"
        }}
    ],
    "output_switches": [
        {{
            "name": "P4_class",
            "iface": "s126-eth2",
            "neuron_id": 126,
            "proc": lambda x: x,
        }},
        {{
            "name": "output_s101",
            "iface": "s126-eth101",
            "neuron_id": 101,
            "proc": lambda x: x/(2**PRECISION) if x < (2**(WORDSIZE - 1)) else (x-(2**WORDSIZE))/(2**PRECISION)
        }}
    ]
}}
WORDSIZE = {WORDSIZE}
PRECISION = {PRECISION}
SLACK = {SLACK}


def eoConverter(x):
    return x/(2**PRECISION) if x < (2**(WORDSIZE - 1)) else (x-(2**WORDSIZE))/(2**PRECISION)


class ANN(Packet):
    fields_desc = [
        BitField("neuron_id", 0, 32),
        '''

for i in range(max_number_of_neurons):
  tester += f"""BitField("data_{i+1}", 0, WORDSIZE), 
        """
     

tester += f"""BitField("run_id", 0, 16),
        BitField("slack", 0, SLACK)
    ]
    
bind_layers(Ether, ANN, type=0x88B5)


def main(cfg):
    # Read input dataset
    input_dataset = pd.read_csv(cfg["input_dataset_filename"])  #.head(50)

    # Create shared queue and packet sniffer to receive ANN outputs
    packet_queue = queue.Queue()
    output_ifs = [x["iface"] for x in cfg["output_switches"]]
    # print(output_ifs)
    sniffers = []
    for output_iface in output_ifs:
        sniffers.append(
            AsyncSniffer(iface=output_iface, prn=lambda x: packet_queue.put(x), stop_filter=lambda x: x.haslayer(ANN) and x[ANN].neuron_id == 0)
        )
    for sniffer in sniffers:
        sniffer.start()

    # Run each of the dataset test cases
    ann_outputs = pd.DataFrame(columns=["""
    

sniff_str = '"s126_id", '
#print(sniff_str)
for i in range(output_layer_nodes):
    sniff_str += f'"s126_data_{i+1}", '

sniff_str += '"s101_id", '
#print(sniff_str)
for i in range(output_layer_nodes):
    sniff_str += f'"s101_data_{i+1}", '
#print(sniff_str)
sniff_str = sniff_str[:-2] # remove the last ", "
#print(sniff_str)
tester += sniff_str

tester += """])
    for tc_id, tc in input_dataset.iterrows():
        # Creat input packets
        # print(f"\\ntc_id, tc: {tc_id}, {tc}")
        input_pkts = []
        
        for switch in cfg["input_switches"]:
            # print(f"switch: {switch}")
            input_pkts.append((
                switch["iface"],
                Ether(dst='ff:ff:ff:ff:ff:ff', src=get_if_hwaddr(switch["iface"])) / ANN(neuron_id=0, """

for i in range(number_of_attributes):
    tester += f"""data_{i+1}=tc["{df_test.columns[i]}"], """

tester += f"""run_id=tc_id)
            ))
# """

tester += f"""
        # print(f"input_pkts: {{input_pkts}}")
        
        # print("entrou primeiro for")
        # Send input packets as many times needed to receive all expected outputs
        n_expected_outputs = len(cfg["output_switches"])
        n_received_outputs = 0
        received_output = False
        tc_outputs = {{}}
        

        while n_received_outputs < n_expected_outputs:
            try:
                # If the queue is empty or this is the first iteration
                if not received_output:
                    # Send input packets
                    for iface, pkt in input_pkts:
                        # print(f"||{{iface}}|| <<{{pkt.show()}}>>")
                        sendp(pkt, iface=iface, verbose=False)
                

                while n_received_outputs < n_expected_outputs:               
                    
                    # Try to get an output packet
                    out_pkt = packet_queue.get(timeout=1)
                    # print(f"out_pkt: {{out_pkt}}")

                    # Check if packet is ANN and an output to the current test case                    
                    if ANN in out_pkt and out_pkt[ANN].run_id == tc_id:
                        if out_pkt[ANN].neuron_id == 126:
                            # print("out_pkt[ANN].neuron_id == 126:")                            
                            tc_outputs["s126_id"] = out_pkt[ANN].neuron_id
"""

for i in range(output_layer_nodes):
    tester += f"""                            tc_outputs["s126_data_{i+1}"] = (out_pkt[ANN].data_{i+1})
"""

tester += f"""
                            received_output = True
                            n_received_outputs = n_received_outputs + 1

                        if out_pkt[ANN].neuron_id == 101:
                            # print("out_pkt[ANN].neuron_id == 101:")                          
                            tc_outputs["s101_id"] = out_pkt[ANN].neuron_id
"""

for i in range(output_layer_nodes):
    tester += f"""                            tc_outputs["s101_data_{i+1}"] = eoConverter(out_pkt[ANN].data_{i+1})
"""

tester += f"""
                            received_output = True
                            n_received_outputs = n_received_outputs + 1

                        # if out_pkt[ANN].neuron_id == 51:
                        #     print("out_pkt[ANN].neuron_id == 51:")    

                        
            
            except queue.Empty as error:
                # If queue was empty, we will send input packet again
                print(f"empty queue")
                print(f"ERROR:{{error}}")
                received_output = False
            except Exception as error:
                # An error here is critical
                print(f"ERROR:{{error}}")
        #print("cheou no fim do 1o while")

        # Add all desired outputs to the dataframe
        # print(f"tc_outputs: {{tc_outputs}}")
        ann_outputs.loc[tc_id] = tc_outputs
        print(end=f"\\r{{tc_id+1}}/{{len(input_dataset)}}")

    for sniffer in sniffers:
        sniffer.stop()

    # Write P4 ANN outputs to a file
    ann_outputs.to_csv(cfg["output_csv_filename"],index=False)


if __name__ == '__main__':
    main(cfg)"""



print(tester)
f = open(tester_file_title, "w")
f.write(tester)
f.close()


import sys
import socket
import struct
import re
import math
import json
import pandas as pd
import numpy as np
import threading
import queue
from scapy.all import *

# Creating the config dict in loco, maybe best would be to be in a file
cfg = {
    "input_dataset_filename": "csv-files-input/df-test-stream-video.csv",
    "output_csv_filename": "csv-files-output/p4-out-stream-video-32x32x4-q8_8.csv",
    "input_switches": [
        {
            "name": "s1",
            "iface": "s1-eth1",
            "id": "s1"
        }
    ],
    "output_switches": [
        {
            "name": "P4_class",
            "iface": "s126-eth2",
            "neuron_id": 126,
            "proc": lambda x: x,
        },
        {
            "name": "output_s101",
            "iface": "s126-eth101",
            "neuron_id": 101,
            "proc": lambda x: x/(2**PRECISION) if x < (2**(WORDSIZE - 1)) else (x-(2**WORDSIZE))/(2**PRECISION)
        }
    ]
}
WORDSIZE = 16
PRECISION = 8
SLACK = 8


def eo

###  File generator for TF predictions
to be compared to P4_prediction in the analiser script

In [9]:
df_test = np.asarray(df_test).astype(np.float32)
tf_predictions_probabilities = model.predict(df_test)
#tf_loss, tf_acc =  model.evaluate(df_test,  target_test, verbose=2, batch_size=batch_size)
tf_predictions = []

for i,x in enumerate(tf_predictions_probabilities):
  #print("i:",i,"x_max:",x.max(),"x:",x)
  j_max = x.argmax()
  tf_predictions.append(j_max)

#conf_m = tf.math.confusion_matrix(target_test,tf_predictions)
#print(conf_m)

df_tf = pd.DataFrame(tf_predictions_probabilities)
df_tf.insert(loc = 0, column = "tf_predictions", value = tf_predictions)
df_tf.to_csv(title_for_analiser, index=False)
print(df_tf)

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
     tf_predictions             0             1             2             3
0                 3 -8.053028e+05 -4.258301e+05 -3.349912e+05  8.204253e+05
1                 3 -9.567399e+07 -9.677357e+07 -5.167484e+07  1.456586e+08
2                 3 -3.648718e+06 -1.072848e+07 -3.293808e+06  1.093402e+07
3                 3  3.199029e+04 -6.907678e+04 -1.424125e+04  4.228739e+04
4                 3 -2.770791e+03 -2.824599e+03 -3.615648e+02  3.223813e+03
..              ...           ...           ...           ...           ...
728               3  1.640594e+05 -4.765028e+05 -2.375156e+05  4.220601e+05
729               3 -1.434183e+04 -2.319051e+04 -1.367148e+04  3.342960e+04
730               3  3.074054e+05 -8.286111e+05 -1.236486e+05  5.150652e+05
731               3 -1.345149e+06 -1.071148e+06 -5.013093e+05  1.619822e+06
732               3  4.438315e+05 -9.454206e+05 -1.909657e+05  5.713934e+05

[733 rows x 5 columns]


### Topology file generator for sx

In [10]:
# print(f"weight: {weights}, norm_mean:{norm_mean}, norm_mean_fp:{norm_mean_fp}, norm_mean_c2:{norm_mean_c2}")
# print(f"bias: {biases}, norm_std:{norm_std}, norm_std_fp:{norm_std_fp}, norm_std_c2:{norm_std_c2}")

In [11]:
# could make all this processing using the same auxiliar value but I choose not, for debug and visualization reasons.
norm_mean = np.zeros(number_of_attributes)
norm_std = np.zeros(number_of_attributes)
norm_mean_fp = np.zeros(number_of_attributes)
norm_std_fp = np.zeros(number_of_attributes)
norm_mean_c2 = np.zeros(number_of_attributes)
norm_std_c2 = np.zeros(number_of_attributes)
weights = norm_layer_weights
biases = norm_layer_biases
act_params = {}

# conversion to fixed point notation Qint.frac
for i in range(number_of_attributes):
  print(f"i:{i}")
  norm_mean[i] = -(weights[i])  
  norm_std[i] = (math.sqrt(biases[i])) ** -1  
  print(f"norm_mean:{norm_mean[i]}")  
    
  norm_mean_fp[i] = int(norm_mean[i] * (2**PRECISION))    
  norm_std_fp[i] = int(norm_std[i] * (2**PRECISION))
  print(f"norm_mean_fp:{norm_mean_fp[i]}")  
    
  norm_mean_c2[i] = num_to_c2(norm_mean_fp[i], n_bits)
  #norm_mean_c2[i] = abs(norm_mean_c2[i])
  norm_std_c2[i] = num_to_c2(norm_std_fp[i], n_bits)
  print(f"norm_mean_c2:{norm_mean_c2[i]}")
    
  act_params[f"neuron_{i + 1}_mean"] = int(f"{norm_mean_c2[i]:.0f}")
  act_params[f"neuron_{i + 1}_std"] = int(f"{norm_std_c2[i]:.0f}")

#print(f"weight: {weights}, norm_mean:{norm_mean}, norm_mean_fp:{norm_mean_fp}, norm_mean_c2:{norm_mean_c2}")
#print(f"bias: {biases}, norm_std:{norm_std}, norm_std_fp:{norm_std_fp}, norm_std_c2:{norm_std_c2}")

#number of norm_layer_nodes = number_of_attributes
for i in range(number_of_input_switches):
  sx_id = 1+i
  sx_entries = {
    "target": "bmv2",
    "p4info": "build/ANN.p4.p4info.txt",
    "bmv2_json": "build/ANN.json",
    "table_entries": [
      {
        "table": "MyIngress.ann_forward",
        "match": {
          "standard_metadata.ingress_port": 1
        },
        "action_name": "MyIngress.mcast",
        "action_params": {
          "mgroup": sx_id
        }
      },
      {
        "table": "MyIngress.tab_neuron_id",
        "default_action": True,
        "action_name": "MyIngress.set_neuron_id",
        "action_params": {
          "neuron_id": sx_id
        }
      },
      {
        "table": "MyIngress.tab_n_expected_stimuli",
        "default_action": True,
        "action_name": "MyIngress.set_n_expected_stimuli",
        "action_params": {
          "n_expected_stimuli": 1
        }
      },
      {
        "table": "MyIngress.tab_expected_stimuli",
        "default_action": True,
        "action_name": "MyIngress.set_expected_stimuli",
        "action_params": {
          "expected_stimuli": 1
        }
      },
      {
        "table": "MyIngress.tab_agg_func",
        "default_action": True,
        "action_name": "MyIngress.set_agg_func",
        "action_params": {
          "agg_func": 5
        }
      },
      {
        "table": "MyIngress.tab_activation_func",
        "default_action": True,
        "action_name": "MyIngress.set_activation_func",
        "action_params": {
          "activation_func": 2
        }
      },
      {
        "table": "MyIngress.tab_norm_mean_std",
        "default_action": True,
        "action_name": "MyIngress.set_norm_mean_std",
        "action_params": act_params
      }
    ],
    "multicast_group_entries":[
      {
        "multicast_group_id" : sx_id,
        "replicas": []
      }
    ]
  }
  for h in range(number_of_hidden_switches):
    sx_entries["multicast_group_entries"][0]["replicas"].append(
      {
        "egress_port" :51+h,
        "instance": 1
      }
    )
  sx_file_title = f"../p4-vm/{use_case}-{topo_str}-q{PRECISION}-{INTEGER}/code/topology/s{sx_id}-runtime.json"
  print(sx_entries["table_entries"])  
  with open(sx_file_title, "w") as f:
    json.dump(sx_entries, f, indent = 2)

i:0
norm_mean:-0.9683189392089844
norm_mean_fp:-247.0
norm_mean_c2:65289.0
i:1
norm_mean:-0.3008943498134613
norm_mean_fp:-77.0
norm_mean_c2:65459.0
i:2
norm_mean:-0.39442169666290283
norm_mean_fp:-100.0
norm_mean_c2:65436.0
i:3
norm_mean:-0.977410078048706
norm_mean_fp:-250.0
norm_mean_c2:65286.0
i:4
norm_mean:-0.42746701836586
norm_mean_fp:-109.0
norm_mean_c2:65427.0
i:5
norm_mean:-0.32545095682144165
norm_mean_fp:-83.0
norm_mean_c2:65453.0
i:6
norm_mean:-1.8325263261795044
norm_mean_fp:-469.0
norm_mean_c2:65067.0
i:7
norm_mean:-1.2299721240997314
norm_mean_fp:-314.0
norm_mean_c2:65222.0
i:8
norm_mean:-0.5115962028503418
norm_mean_fp:-130.0
norm_mean_c2:65406.0
i:9
norm_mean:-0.6342276930809021
norm_mean_fp:-162.0
norm_mean_c2:65374.0
i:10
norm_mean:-0.5337274670600891
norm_mean_fp:-136.0
norm_mean_c2:65400.0
i:11
norm_mean:-0.031074730679392815
norm_mean_fp:-7.0
norm_mean_c2:65529.0
i:12
norm_mean:-3.3500943183898926
norm_mean_fp:-857.0
norm_mean_c2:64679.0
i:13
norm_mean:-0.6263453

### Topology file generator for s5x

In [12]:
# Hidden Layer Switches

# Calculate bias values using Fixed Point (Qinteger.precision) and Two's Complement (C2) and for
bias_params = {}
biases = hidden_layer_biases
fp_bias_value = np.zeros(hidden_layer_nodes)
c2_bias_value = np.zeros(hidden_layer_nodes)
for i in range(hidden_layer_nodes):
  fp_bias_value[i] = int(biases[i] * (2**PRECISION))
  c2_bias_value[i] = num_to_c2(fp_bias_value[i], n_bits)
  bias_params[f"neuron_{i + 1}_bias"] = int(f"{c2_bias_value[i]:.0f}")
# print(c2_bias_value)

# Calculate weights
weights_params = {}
weights = hidden_layer_weights
fp_value = np.zeros((number_of_attributes,hidden_layer_nodes))
c2_value = np.zeros((number_of_attributes,hidden_layer_nodes))
for i in range(hidden_layer_nodes):
  for j in range(number_of_attributes):
    fp_value[j][i] = int(weights[j][i] * (2**PRECISION))
    c2_value[j][i] = num_to_c2(fp_value[j][i], n_bits)
    weights_params[f"n2n_{i + 1}_weight_{j + 1}"] = int(f"{c2_value[j][i]:.0f}")
#print(c2_value)

# Calculate expected stimuli
expected_stimuli = sum([1<<(i + 1) for i in range(number_of_attributes)])

# Create tables and actions names
table_neuron_bias_str = f"MyIngress.tab_neuron_bias_{hidden_layer_nodes}_neurons"
action_neuron_bias_str = f"MyIngress.set_neuron_bias_{hidden_layer_nodes}_neurons"
table_neuron_weight_str = f"MyIngress.tab_n2n_weight_{number_of_attributes}_to_{hidden_layer_nodes}_neurons"
action_neuron_weight_str = f"MyIngress.set_n2n_weight_{number_of_attributes}_to_{hidden_layer_nodes}_neurons"

# Pass information to switch tables
for i in range(number_of_hidden_switches):
  s5x_id = 51+i
  s5x_entries = {
    "target": "bmv2",
    "p4info": "build/ANN.p4.p4info.txt",
    "bmv2_json": "build/ANN.json",
    "table_entries": [
      {
        "table": "MyIngress.tab_neuron_id",
        "default_action": True,
        "action_name": "MyIngress.set_neuron_id",
        "action_params": {
          "neuron_id": s5x_id
        }
      },
      {
        "table": "MyIngress.tab_n_expected_stimuli",
        "default_action": True,
        "action_name": "MyIngress.set_n_expected_stimuli",
        "action_params": {
          "n_expected_stimuli": number_of_input_switches
        }
      },
      {
        "table": "MyIngress.tab_expected_stimuli",
        "default_action": True,
        "action_name": "MyIngress.set_expected_stimuli",
        "action_params": {
          "expected_stimuli": expected_stimuli
        }
      },
      {
        "table": "MyIngress.tab_agg_func",
        "default_action": True,
        "action_name": "MyIngress.set_agg_func",
        "action_params": {
          "agg_func": 1
        }
      },
      {
        "table": "MyIngress.tab_activation_func",
        "default_action": True,
        "action_name": "MyIngress.set_activation_func",
        "action_params": {
          "activation_func": 3
        }
      },
      {
        "table": table_neuron_bias_str,
        "default_action": True,
        "action_name": action_neuron_bias_str,
        "action_params": bias_params
      },
      {
        "table": table_neuron_weight_str,
        "match": {
          "hdr.ann.neuron_id": i+1
        },
        "action_name": action_neuron_weight_str,
        "action_params": weights_params
      }
    ],
    "multicast_group_entries":[
      {
        "multicast_group_id" : s5x_id,
        "replicas":[]
      }
    ]
  }
  for j in range(number_of_input_switches):
    s5x_entries["table_entries"].append(
      {
        "table": "MyIngress.ann_forward",
        "match": {
          "standard_metadata.ingress_port": j+1
        },
        "action_name": "MyIngress.mcast",
        "action_params": {
          "mgroup": s5x_id
        }
      }
    )
  for h in range(number_of_output_switches):
    s5x_entries["multicast_group_entries"][0]["replicas"].append(
      {
        "egress_port" :101+h,
        "instance": 1
      }
    )
  s5x_file_title = f"../p4-vm/{use_case}-{topo_str}-q{PRECISION}-{INTEGER}/code/topology/s{s5x_id}-runtime.json"
  with open(s5x_file_title, "w") as f:
    json.dump(s5x_entries, f, indent = 2)
      
print(json.dumps(s5x_entries, indent = 2))

{
  "target": "bmv2",
  "p4info": "build/ANN.p4.p4info.txt",
  "bmv2_json": "build/ANN.json",
  "table_entries": [
    {
      "table": "MyIngress.tab_neuron_id",
      "default_action": true,
      "action_name": "MyIngress.set_neuron_id",
      "action_params": {
        "neuron_id": 51
      }
    },
    {
      "table": "MyIngress.tab_n_expected_stimuli",
      "default_action": true,
      "action_name": "MyIngress.set_n_expected_stimuli",
      "action_params": {
        "n_expected_stimuli": 1
      }
    },
    {
      "table": "MyIngress.tab_expected_stimuli",
      "default_action": true,
      "action_name": "MyIngress.set_expected_stimuli",
      "action_params": {
        "expected_stimuli": 8589934590
      }
    },
    {
      "table": "MyIngress.tab_agg_func",
      "default_action": true,
      "action_name": "MyIngress.set_agg_func",
      "action_params": {
        "agg_func": 1
      }
    },
    {
      "table": "MyIngress.tab_activation_func",
      "default_actio

### Topology file generator for s1xx

In [13]:
# Output Layer Switches

# Calculate bias values using Fixed Point (Qinteger.precision) and Two's Complement (C2)
bias_params = {}
biases = out_layer_biases
fp_bias_value = np.zeros(output_layer_nodes)
c2_bias_value = np.zeros(output_layer_nodes)
for i in range(output_layer_nodes):
  fp_bias_value[i] = int(biases[i] * (2**PRECISION))
  c2_bias_value[i] = num_to_c2(fp_bias_value[i], n_bits)
  bias_params[f"neuron_{i + 1}_bias"] = int(f"{c2_bias_value[i]:.0f}")
#print(c2_bias_value)

weights_params = {}
weights = out_layer_weights
fp_value = np.zeros((hidden_layer_nodes,output_layer_nodes))
c2_value = np.zeros((hidden_layer_nodes,output_layer_nodes))
for i in range(output_layer_nodes):
  for j in range(hidden_layer_nodes):
    fp_value[j][i] = int(weights[j][i] * (2**PRECISION))
    c2_value[j][i] = num_to_c2(fp_value[j][i], n_bits)
    weights_params[f"n2n_{i + 1}_weight_{j + 1}"] = int(f"{c2_value[j][i]:.0f}")
#print(c2_value)

# Calculate expected stimuli
expected_stimuli = sum([1<<(i + 51) for i in range(hidden_layer_nodes)])

# Create tables and actions names
table_neuron_bias_str = f"MyIngress.tab_neuron_bias_{output_layer_nodes}_neurons"
action_neuron_bias_str = f"MyIngress.set_neuron_bias_{output_layer_nodes}_neurons"
table_neuron_weight_str = f"MyIngress.tab_n2n_weight_{hidden_layer_nodes}_to_{output_layer_nodes}_neurons"
action_neuron_weight_str = f"MyIngress.set_n2n_weight_{hidden_layer_nodes}_to_{output_layer_nodes}_neurons"

# Pass information to switch tables
for i in range(number_of_output_switches):
  s1xx_id = 101+i
  s1xx_entries = {
    "target": "bmv2",
    "p4info": "build/ANN.p4.p4info.txt",
    "bmv2_json": "build/ANN.json",
    "table_entries": [
      {
        "table": "MyIngress.tab_neuron_id",
        "default_action": True,
        "action_name": "MyIngress.set_neuron_id",
        "action_params": {
          "neuron_id": s1xx_id
        }
      },
      {
        "table": "MyIngress.tab_n_expected_stimuli",
        "default_action": True,
        "action_name": "MyIngress.set_n_expected_stimuli",
        "action_params": {
          "n_expected_stimuli": number_of_hidden_switches
        }
      },
      {
        "table": "MyIngress.tab_expected_stimuli",
        "default_action": True,
        "action_name": "MyIngress.set_expected_stimuli",
        "action_params": {
          "expected_stimuli": expected_stimuli

        }
      },
      {
        "table": "MyIngress.tab_agg_func",
        "default_action": True,
        "action_name": "MyIngress.set_agg_func",
        "action_params": {
          "agg_func": 6
        }
      },
      {
        "table": "MyIngress.tab_activation_func",
        "default_action": True,
        "action_name": "MyIngress.set_activation_func",
        "action_params": {
          "activation_func": 2
        }
      },
      {
        "table": table_neuron_bias_str,
        "default_action": True,
        "action_name": action_neuron_bias_str,
        "action_params": bias_params
      },
      {
        "table": table_neuron_weight_str,
        "match": {
          "hdr.ann.neuron_id": i+51
        },
        "action_name": action_neuron_weight_str,
        "action_params": weights_params
      }
    ],
    "multicast_group_entries":[
      {
        "multicast_group_id" : s1xx_id,
        "replicas":[
          {
            "egress_port" :126,
            "instance": 1
          }
        ]
      }
    ]
  }
  for j in range(number_of_input_switches):
    s1xx_entries["table_entries"].append(
      {
        "table": "MyIngress.ann_forward",
        "match": {
          "standard_metadata.ingress_port": j+51
        },
        "action_name": "MyIngress.mcast",
        "action_params": {
          "mgroup": s1xx_id
        }
      }
    )
  s1xx_file_title = f"../p4-vm/{use_case}-{topo_str}-q{PRECISION}-{INTEGER}/code/topology/s{s1xx_id}-runtime.json"
  with open(s1xx_file_title, "w") as f:
    json.dump(s1xx_entries, f, indent = 2)
      
print(json.dumps(s1xx_entries, indent = 2))

{
  "target": "bmv2",
  "p4info": "build/ANN.p4.p4info.txt",
  "bmv2_json": "build/ANN.json",
  "table_entries": [
    {
      "table": "MyIngress.tab_neuron_id",
      "default_action": true,
      "action_name": "MyIngress.set_neuron_id",
      "action_params": {
        "neuron_id": 101
      }
    },
    {
      "table": "MyIngress.tab_n_expected_stimuli",
      "default_action": true,
      "action_name": "MyIngress.set_n_expected_stimuli",
      "action_params": {
        "n_expected_stimuli": 1
      }
    },
    {
      "table": "MyIngress.tab_expected_stimuli",
      "default_action": true,
      "action_name": "MyIngress.set_expected_stimuli",
      "action_params": {
        "expected_stimuli": 9671406554665233583964160
      }
    },
    {
      "table": "MyIngress.tab_agg_func",
      "default_action": true,
      "action_name": "MyIngress.set_agg_func",
      "action_params": {
        "agg_func": 6
      }
    },
    {
      "table": "MyIngress.tab_activation_func",
    

### P4 File Generation: defines, headers, parser, checksum

In [14]:
headers = f"""
/* -*- P4_16 -*- */
#include <core.p4>
#include <v1model.p4>

const bit<16> TYPE_IPV4 = 0x800;

#define ET_ANN 0x88B5

#define FUNC_WEIGHTED_SUM_{number_of_attributes}_TO_{hidden_layer_nodes} 1
#define FUNC_IDENTITY 2
#define FUNC_RELU 3
#define FUNC_ARGMAX 4
#define FUNC_NORMALIZATION 5
#define FUNC_WEIGHTED_SUM_{hidden_layer_nodes}_TO_{output_layer_nodes} 6

#define PRECISION {PRECISION}
#define WORDSIZE {WORDSIZE}
#define D_WORDSIZE {D_WORDSIZE}
#define SLACK {SLACK}

/*************************************************************************
*********************** H E A D E R S  ***********************************
*************************************************************************/

typedef bit<9>  egressSpec_t;
typedef bit<48> macAddr_t;
typedef bit<32> ip4Addr_t;

header ethernet_t{{
    macAddr_t dstAddr;
    macAddr_t srcAddr;
    bit<16>   etherType;
}}

header ann_t{{
    bit<32> neuron_id;
"""
for i in range(max_number_of_neurons):
  headers += f"    bit<WORDSIZE> data_{i+1};\n"
headers += """    bit<16> run_id;
    bit<SLACK> slack;
}

struct metadata{
    bit<32> neuron_id;            // temporarily stores the ID of the neuron in this switch.
    bit<32> n_expected_stimuli;   // temporarily stores the number of expected stimuli by the neuron in a single ANN run.
    bit<32> n_received_stimuli;   // temporarily stores the number of stimuli already received by the neuron in the current ANN run.
    bit<128> expected_stimuli;    // temporarily stores a bitstring that indicates from which neurons is the neuron expected to receive stimuli. For example, if the bitstring has value 0b1010, the neuron is expected to receive stimuli from neurons with ID 1 and 3, but not from IDs 0 and 2.
    bit<128> received_stimuli;    // temporarily stores a bitstring that indicates from which neurons is the neuron already received stimuli in the current ANN run.

    bit<32> agg_func;
    bit<32> activation_func;
    bit<16> run_id;

    // Stores the data to be processed and fowarded
"""
for i in range(max_number_of_neurons):
  headers += f"    bit<WORDSIZE> neuron_{i+1}_data;\n"

headers += f"\n    // Agg func = normalization\n"
for i in range(number_of_attributes):
  headers += f"    bit<WORDSIZE> neuron_{i+1}_mean;\n"
  headers += f"    bit<WORDSIZE> neuron_{i+1}_std;\n"

headers += f"\n    // Agg func = weighted sum\n"
for i in range(hidden_layer_nodes):
  headers += f"    bit<WORDSIZE> neuron_{i+1}_bias;\n"

for i in range(max(hidden_layer_nodes,output_layer_nodes)):
  for j in range(max(number_of_attributes,hidden_layer_nodes)):
    headers += f"    bit<WORDSIZE> n2n_{i+1}_weight_{j+1};\n"

headers += """
// Agg func = argmax
    bit<WORDSIZE> neuron_max_value;
}

struct headers{
    ethernet_t   ethernet;
    ann_t   ann;
}

/*************************************************************************
*********************** P A R S E R  ***********************************
*************************************************************************/

parser MyParser(packet_in packet,
            out headers hdr,
            inout metadata meta,
            inout standard_metadata_t standard_metadata){

    state start{
        transition parse_ethernet;
    }

    state parse_ethernet{
        packet.extract(hdr.ethernet);
        transition select(hdr.ethernet.etherType){
           ET_ANN: parse_ann;
           default: accept;
        }
    }

    state parse_ann{
      packet.extract(hdr.ann);
      transition accept;
    }
}

/*************************************************************************
************   C H E C K S U M    V E R I F I C A T I O N   *************
*************************************************************************/

control MyVerifyChecksum(inout headers hdr, inout metadata meta){
    apply {  }
}
"""
print(headers)


/* -*- P4_16 -*- */
#include <core.p4>
#include <v1model.p4>

const bit<16> TYPE_IPV4 = 0x800;

#define ET_ANN 0x88B5

#define FUNC_WEIGHTED_SUM_32_TO_32 1
#define FUNC_IDENTITY 2
#define FUNC_RELU 3
#define FUNC_ARGMAX 4
#define FUNC_NORMALIZATION 5
#define FUNC_WEIGHTED_SUM_32_TO_4 6

#define PRECISION 8
#define WORDSIZE 16
#define D_WORDSIZE 32
#define SLACK 8

/*************************************************************************
*********************** H E A D E R S  ***********************************
*************************************************************************/

typedef bit<9>  egressSpec_t;
typedef bit<48> macAddr_t;
typedef bit<32> ip4Addr_t;

header ethernet_t{
    macAddr_t dstAddr;
    macAddr_t srcAddr;
    bit<16>   etherType;
}

header ann_t{
    bit<32> neuron_id;
    bit<WORDSIZE> data_1;
    bit<WORDSIZE> data_2;
    bit<WORDSIZE> data_3;
    bit<WORDSIZE> data_4;
    bit<WORDSIZE> data_5;
    bit<WORDSIZE> data_6;
    bit<WORDSIZE> data_7;
    bit<W

### Ingress, tables and actions

In [15]:
ingress = """
/*************************************************************************
**************  I N G R E S S   P R O C E S S I N G   *******************
*************************************************************************/

control MyIngress(inout headers hdr,
                inout metadata meta,
                inout standard_metadata_t standard_metadata) {

    register<bit<32>>(1) reg_n_received_stimuli;
    register<bit<128>>(1) reg_received_stimuli;
"""
for i in range(max_number_of_neurons):
  ingress += f"    register<bit<WORDSIZE>>(1) reg_neuron_{i+1}_data;\n"
ingress +="""
    register<bit<WORDSIZE>>(1) reg_neuron_max_value;
    register<bit<16>>(1) reg_run_id;
    action drop(){
    mark_to_drop(standard_metadata);
}

action mcast(bit<16> mgroup){
    standard_metadata.mcast_grp = mgroup;
}

table ann_forward{
    key = {
        standard_metadata.ingress_port: exact;
    }
    actions = {
        mcast;
        drop;
    }
    size = 1024;
    default_action = drop();
}

action set_neuron_id(bit<32> neuron_id){
    meta.neuron_id = neuron_id;
}

table tab_neuron_id{
    actions = {
        set_neuron_id;
    }
    size = 1;
}

action set_n_expected_stimuli(bit<32> n_expected_stimuli){
    meta.n_expected_stimuli = n_expected_stimuli;
}

table tab_n_expected_stimuli{
    actions = {
        set_n_expected_stimuli;
    }
    size = 1;
}

action set_expected_stimuli(bit<128> expected_stimuli){
    meta.expected_stimuli = expected_stimuli;
}

table tab_expected_stimuli{
    actions = {
        set_expected_stimuli;
    }
    size = 1;
}

action set_agg_func(bit<32> agg_func){
    meta.agg_func = agg_func;
}

table tab_agg_func{
    actions = {
        set_agg_func;
    }
    size = 1;
}


"""

ingress += f"action set_neuron_bias_{hidden_layer_nodes}_neurons("

for i in range(hidden_layer_nodes):
  ingress += f"bit<WORDSIZE> neuron_{i+1}_bias, "
ingress = ingress[:-2] # remove the last ", "
ingress += "){\n"

for i in range(hidden_layer_nodes):
  ingress += f"   meta.neuron_{i+1}_bias = neuron_{i+1}_bias;\n"
ingress += f"""}}

table tab_neuron_bias_{hidden_layer_nodes}_neurons{{
    actions = {{
        set_neuron_bias_{hidden_layer_nodes}_neurons;
    }}
    size = 1;
}}
"""
#tem que botar um if pra ver se o hidden é diferente do output
ingress += f"action set_neuron_bias_{output_layer_nodes}_neurons("
for i in range(output_layer_nodes):
  ingress += f"bit<WORDSIZE> neuron_{i+1}_bias, "
ingress = ingress[:-2] # remove the last ", "
ingress += "){\n"

for i in range(output_layer_nodes):
  ingress += f"   meta.neuron_{i+1}_bias = neuron_{i+1}_bias;\n"
ingress += f"""}}

table tab_neuron_bias_{output_layer_nodes}_neurons{{
    actions = {{
        set_neuron_bias_{output_layer_nodes}_neurons;
    }}
    size = 1;
}}

action set_n2n_weight_{number_of_attributes}_to_{hidden_layer_nodes}_neurons("""
for i in range(hidden_layer_nodes):
  for j in range(number_of_attributes):
    ingress += f"bit<WORDSIZE> n2n_{i+1}_weight_{j+1}, "
ingress = ingress[:-2] # remove the last ", "

ingress += "){\n"
for i in range(hidden_layer_nodes):
  for j in range(number_of_attributes):
    ingress += f"   meta.n2n_{i+1}_weight_{j+1} = n2n_{i+1}_weight_{j+1};\n"
ingress += f"""}}

table tab_n2n_weight_{number_of_attributes}_to_{hidden_layer_nodes}_neurons{{
    key = {{
        hdr.ann.neuron_id: exact;
    }}
    actions = {{
        set_n2n_weight_{number_of_attributes}_to_{hidden_layer_nodes}_neurons;
    }}
    size = 256;
}}

action set_n2n_weight_{hidden_layer_nodes}_to_{output_layer_nodes}_neurons("""
for i in range(output_layer_nodes):
  for j in range(hidden_layer_nodes):
    ingress += f"bit<WORDSIZE> n2n_{i+1}_weight_{j+1}, "
ingress = ingress[:-2] # remove the last ", "

ingress += "){\n"
for i in range(output_layer_nodes):
  for j in range(hidden_layer_nodes):
    ingress += f"   meta.n2n_{i+1}_weight_{j+1} = n2n_{i+1}_weight_{j+1};\n"
ingress += f"""}}

table tab_n2n_weight_{hidden_layer_nodes}_to_{output_layer_nodes}_neurons{{
    key = {{
        hdr.ann.neuron_id: exact;
    }}
    actions = {{
        set_n2n_weight_{hidden_layer_nodes}_to_{output_layer_nodes}_neurons;
    }}
    size = 256;
}}

action set_norm_mean_std("""
for i in range(number_of_attributes):
  ingress += f"bit<WORDSIZE> neuron_{i+1}_mean, bit<WORDSIZE> neuron_{i+1}_std, "
ingress = ingress[:-2] # remove the last ", "
ingress += "){\n"
for i in range(number_of_attributes):
  ingress += f"   meta.neuron_{i+1}_mean = neuron_{i+1}_mean;\n"
  ingress += f"   meta.neuron_{i+1}_std = neuron_{i+1}_std;\n"
ingress += """
}

table tab_norm_mean_std{
    actions = {
        set_norm_mean_std;
    }
    size = 1;
}

action set_activation_func(bit<32> activation_func){
    meta.activation_func = activation_func;
}

table tab_activation_func{
    actions = {
        set_activation_func;
    }
    size = 1;
}
"""
print(ingress)


/*************************************************************************
**************  I N G R E S S   P R O C E S S I N G   *******************
*************************************************************************/

control MyIngress(inout headers hdr,
                inout metadata meta,
                inout standard_metadata_t standard_metadata) {

    register<bit<32>>(1) reg_n_received_stimuli;
    register<bit<128>>(1) reg_received_stimuli;
    register<bit<WORDSIZE>>(1) reg_neuron_1_data;
    register<bit<WORDSIZE>>(1) reg_neuron_2_data;
    register<bit<WORDSIZE>>(1) reg_neuron_3_data;
    register<bit<WORDSIZE>>(1) reg_neuron_4_data;
    register<bit<WORDSIZE>>(1) reg_neuron_5_data;
    register<bit<WORDSIZE>>(1) reg_neuron_6_data;
    register<bit<WORDSIZE>>(1) reg_neuron_7_data;
    register<bit<WORDSIZE>>(1) reg_neuron_8_data;
    register<bit<WORDSIZE>>(1) reg_neuron_9_data;
    register<bit<WORDSIZE>>(1) reg_neuron_10_data;
    register<bit<WORDSIZE>>(1) reg_neu

### Apply

In [16]:
apply = """
apply {
    if(hdr.ann.isValid()){                                   // If the ANN header is present in the packet
        reg_run_id.read(meta.run_id, 0);
        if(hdr.ann.run_id != meta.run_id){                   // If the run_id in the receiving differs from the stored run_id, reset the received stimuli so we don't mix up data
            reg_run_id.write(0, hdr.ann.run_id);
            reg_received_stimuli.write(0, 0);
            reg_n_received_stimuli.write(0, 0);
        }
        tab_expected_stimuli.apply();                         // Get the bitstring of expected stimuli and store in the MD field
        reg_received_stimuli.read(meta.received_stimuli, 0);  // Get the bitstring of received stimuli and store in the MD field

        // Declare and compute the value of a variable that indicates whether the stimulus in the packet is expected
        bit<128> expected = meta.expected_stimuli & ((bit<128>) 1 << (bit<8>) hdr.ann.neuron_id); // the bit shift and & operator enable us to do the checking.
        // Declare and compute the value of a variable that indicates whether the stimulus in the packet has been received
        bit<128> received = meta.received_stimuli & ((bit<128>) 1 << (bit<8>) hdr.ann.neuron_id);

        // Check if the stimulus is expected and was not yet received
        if((expected > (bit<128>) 0) && (received == (bit<128>) 0)){
            meta.received_stimuli = meta.received_stimuli | ((bit<128>) 1 << (bit<8>) hdr.ann.neuron_id);
            reg_received_stimuli.write(0, meta.received_stimuli);
            // Load n_received_stimuli from register, increment it, and write back
            reg_n_received_stimuli.read(meta.n_received_stimuli, 0);
            meta.n_received_stimuli = meta.n_received_stimuli + 1;
            reg_n_received_stimuli.write(0, meta.n_received_stimuli);
            // Set the register(s) storing the neuron aggregation and bias function
            tab_agg_func.apply();
"""


apply += f"            tab_neuron_bias_{hidden_layer_nodes}_neurons.apply();\n"
apply += f"            tab_neuron_bias_{output_layer_nodes}_neurons.apply();"


### Normalization

In [17]:
normalization = """
            //Calculate the aggregation funciton

            if(meta.agg_func == FUNC_NORMALIZATION){
                // normalized_value = (raw_value - weight) / sqrt(biases)
                // since there's no subtraction nor division in P4, must adequate the formula to
                // normalized_value = (raw_value + (-weight) * (sqrt(bias)) ** -1

                // A NOTE ON SIGN EXTENSION: When we extend the number of bits of a negative number, we must also extend the signal to keep the correctness.
                // Example	-Corect: positive, no need for sign extension	w: 0001 -> dw: 0000 0001
                //			-Corect: negative, with sign extension 			w: 1110 -> dw: 1111 1110
                //			-WRONG:  negative, without sign extension 		w: 1110 -> dw: 0000 1110

                tab_norm_mean_std.apply(); // Load weight (mean) and bias (std)
"""
for i in range(number_of_attributes):
  normalization += f"""
                // Neuron {i+1}:
                bit<WORDSIZE> operand_a{i+1} = hdr.ann.data_{i+1} << PRECISION; // Pass the values to registers to be able to operate them. To load the input data, which are integers, need to shift left to adequate them to FP notation Q.INT.FRAC. TO_DO need special treatment to NEGATIVE INPUT DATA!!!
                bit<WORDSIZE> operand_b{i+1} = meta.neuron_{i+1}_mean; // Load normalization means (= weights)
                bit<WORDSIZE> sum_result_{i+1} = operand_a{i+1} + operand_b{i+1}; // Compute the sum
                bit<D_WORDSIZE> sum_result_{i+1}_dw = (bit<D_WORDSIZE>) sum_result_{i+1}; // Need to double the wordsize to store the multiplication result
                if((sum_result_{i+1}_dw & (1 << (WORDSIZE-1))) > 0){{                            // negative number
                    sum_result_{i+1}_dw = ((1 << D_WORDSIZE) - (1 << WORDSIZE)) + (bit<D_WORDSIZE>) sum_result_{i+1}_dw;
                }}
                bit<D_WORDSIZE> operand_c{i+1} = (bit<D_WORDSIZE>) meta.neuron_{i+1}_std;  // Load normalization std (= bias)
                // Sign extension
                if((operand_c{i+1} & (1 << (WORDSIZE-1))) > 0){{                                // negative number
                    operand_c{i+1} = ((1 << D_WORDSIZE) - (1 << WORDSIZE)) + (bit<D_WORDSIZE>) operand_c{i+1};
                }}
                bit<D_WORDSIZE> norm_result_{i+1} = ((sum_result_{i+1}_dw * operand_c{i+1}) >> PRECISION);
                meta.neuron_{i+1}_data = (bit<WORDSIZE>) norm_result_{i+1}; // Resize the data to wordsize to be fowarded
                reg_neuron_{i+1}_data.write(0, meta.neuron_{i+1}_data);  // Store the value to be fowarded
"""
normalization += "            }"
print(normalization)


            //Calculate the aggregation funciton

            if(meta.agg_func == FUNC_NORMALIZATION){
                // normalized_value = (raw_value - weight) / sqrt(biases)
                // since there's no subtraction nor division in P4, must adequate the formula to
                // normalized_value = (raw_value + (-weight) * (sqrt(bias)) ** -1

                // A NOTE ON SIGN EXTENSION: When we extend the number of bits of a negative number, we must also extend the signal to keep the correctness.
                // Example	-Corect: positive, no need for sign extension	w: 0001 -> dw: 0000 0001
                //			-Corect: negative, with sign extension 			w: 1110 -> dw: 1111 1110
                //			-WRONG:  negative, without sign extension 		w: 1110 -> dw: 0000 1110

                tab_norm_mean_std.apply(); // Load weight (mean) and bias (std)

                // Neuron 1:
                bit<WORDSIZE> operand_a1 = hdr.ann.data_1 << PRECISION; // Pass the values to regi

### Weighted Sum

In [18]:
def w_sum_gen(previous_layer_nodes, this_layer_nodes):
  w_sum = f"""
            else if(meta.agg_func == FUNC_WEIGHTED_SUM_{previous_layer_nodes}_TO_{this_layer_nodes}){{                // Aggregation Function = weighted sum = bias + Summation_i=1_to_n(data_i * weight_i)
                if(meta.n_received_stimuli == 1){{ // Check if this is the first stimulus in an ANN run
                    // If yes, initialize neuron_data with the neuron bias, the neuron bias is added to the accumulator (neuron_data) just once
"""
  for i in range(this_layer_nodes):
    w_sum += f"                    meta.neuron_{i+1}_data = meta.neuron_{i+1}_bias;\n"
  w_sum += """                }
                else{
                    // If not, read the neuron_data value stored in the register
"""
  for i in range(this_layer_nodes):
    w_sum += f"                    reg_neuron_{i+1}_data.read(meta.neuron_{i+1}_data, 0);\n"
  w_sum += """                }
                """
  w_sum += f"""tab_n2n_weight_{previous_layer_nodes}_to_{this_layer_nodes}_neurons.apply();	// Get the neuron weights
                // Load data and perform sign extension
"""
  for i in range(previous_layer_nodes):
    w_sum += f"""
                bit<D_WORDSIZE> operand_b{i+1} = (bit<D_WORDSIZE>) hdr.ann.data_{i+1};
                // Sign extension
                if((operand_b{i+1} & (1 << (WORDSIZE-1))) > 0){{ // negative number
                    operand_b{i+1} = ((1 << D_WORDSIZE) - (1 << WORDSIZE)) + (bit<D_WORDSIZE>) operand_b{i+1};
                }}"""

  for i in range(this_layer_nodes):
    w_sum += f"""
                // Neuron {i+1}:
                // Pass values to aux variable to be able to operate them
"""
    for j in range(previous_layer_nodes):
      w_sum += f"                bit<D_WORDSIZE> operand_a_{i+1}_{j+1} = (bit<D_WORDSIZE>) meta.n2n_{i+1}_weight_{j+1};\n"
    w_sum += f"                // Sign extension"
    for j in range(previous_layer_nodes):
      w_sum += f"""
                if((operand_a_{i+1}_{j+1} & (1 << (WORDSIZE-1))) > 0){{ // negative number
                    operand_a_{i+1}_{j+1} = ((1 << D_WORDSIZE) - (1 << WORDSIZE)) + (bit<D_WORDSIZE>) operand_a_{i+1}_{j+1};
                }}"""

    w_sum += f"\n                // perform multiplication and shift to transform data back to wordsize\n"
    for j in range(previous_layer_nodes):
      w_sum += f"                bit<D_WORDSIZE> res_{i+1}_{j+1} = ((operand_a_{i+1}_{j+1} * operand_b{j+1}) >> PRECISION);\n"
    w_sum += f"""
                // Compute the summation
                meta.neuron_{i+1}_data = meta.neuron_{i+1}_data"""
    for j in range(previous_layer_nodes):
      w_sum += f" + (bit<WORDSIZE>) res_{i+1}_{j+1}"
    w_sum += f""";
                // Store data to be fowarded
                reg_neuron_{i+1}_data.write(0, meta.neuron_{i+1}_data);
"""
  w_sum += "            }"
  return w_sum

w_sum_str = w_sum_gen(number_of_attributes, hidden_layer_nodes)
w_sum_str += w_sum_gen(hidden_layer_nodes, output_layer_nodes)

print(w_sum_str)


            else if(meta.agg_func == FUNC_WEIGHTED_SUM_32_TO_32){                // Aggregation Function = weighted sum = bias + Summation_i=1_to_n(data_i * weight_i)
                if(meta.n_received_stimuli == 1){ // Check if this is the first stimulus in an ANN run
                    // If yes, initialize neuron_data with the neuron bias, the neuron bias is added to the accumulator (neuron_data) just once
                    meta.neuron_1_data = meta.neuron_1_bias;
                    meta.neuron_2_data = meta.neuron_2_bias;
                    meta.neuron_3_data = meta.neuron_3_bias;
                    meta.neuron_4_data = meta.neuron_4_bias;
                    meta.neuron_5_data = meta.neuron_5_bias;
                    meta.neuron_6_data = meta.neuron_6_bias;
                    meta.neuron_7_data = meta.neuron_7_bias;
                    meta.neuron_8_data = meta.neuron_8_bias;
                    meta.neuron_9_data = meta.neuron_9_bias;
                    meta.neuron_10_d

### Argmax

In [19]:
argmax = """
            else if(meta.agg_func == FUNC_ARGMAX){
                // the data to be fowarded (neuron_1_data) is the ID of the switch with highest value.
                // neuron_2_data is the index of the neuron with highest value inside the same switch.
                // the highest data (neuron_max_value) is kept to be compared by other neurons.
                bit<WORDSIZE> op_a = 0;
                bit<WORDSIZE> op_b = 0;
                bit<1> op_a_sig = 0;
                bit<1> op_b_sig = 0;
                if(meta.n_received_stimuli == 1){
                    // if first stimuli, then assume first data received is the higher, then check the remmaining data against it
                    // Neuron 1
                    meta.neuron_1_data = (bit<WORDSIZE>) hdr.ann.neuron_id;
                    meta.neuron_2_data = 0; // neuron_2_data stores the index of the neuron with highest value within same switch
                    meta.neuron_max_value = hdr.ann.data_1;
"""
for i in range(output_layer_nodes-1):
  argmax += f"""
                    // Neuron {i+2}
                    // Check if data is higher than stored max data
                    op_a = hdr.ann.data_{i+2}; 			// op_a is the data being evaluated if it's higher then the stored one (op_b)
                    op_b = meta.neuron_max_value;		// op_b is the store of max value until now
                    op_a_sig = (bit<1>)(op_a & (1 << (WORDSIZE-1)) > 0);
                    op_b_sig = (bit<1>)(op_b & (1 << (WORDSIZE-1)) > 0);
                    // There are two situation in which op_a is bigger then op_b
                    if((op_a_sig == 0) && (op_b_sig  == 1)){{ // The first: if the op_a is positive and op_b is negative
                        //meta.neuron_1_data = (bit<WORDSIZE>) hdr.ann.neuron_id;
                        meta.neuron_2_data = {i+1};
                        meta.neuron_max_value = hdr.ann.data_{i+2};
                    }} else if(op_a_sig == op_b_sig && op_a > op_b){{ // The second: if the signal is the same, and op_a > op_b
                        //meta.neuron_1_data = (bit<WORDSIZE>) hdr.ann.neuron_id;
                        meta.neuron_2_data = {i+1};
                        meta.neuron_max_value = hdr.ann.data_{i+2};
                    }}"""
argmax += """
                }
                else{
                    // this is wrong!?!? reg_neuron_1_data.read(meta.neuron_1_data, 0);
                    reg_neuron_max_value.read(meta.neuron_max_value, 0);
"""
for i in range(output_layer_nodes):
  argmax += f"""
                    // Neuron {i+1}
                    // Check if data is higher than stored max data
                    op_a = hdr.ann.data_{i+1}; 			// op_a is the data being evaluated if it's higher then the stored one (op_b)
                    op_b = meta.neuron_max_value;		// op_b is the store of max value until now
                    op_a_sig = (bit<1>)(op_a & (1 << (WORDSIZE-1)) > 0);
                    op_b_sig = (bit<1>)(op_b & (1 << (WORDSIZE-1)) > 0);
                    // There are two situation in which op_a is bigger then op_b
                    if((op_a_sig == 0) && (op_b_sig  == 1)){{ // The first: if the op_a is positive and op_b is negative
                        //meta.neuron_1_data = (bit<WORDSIZE>) hdr.ann.neuron_id;
                        meta.neuron_2_data = {i};
                        meta.neuron_max_value = hdr.ann.data_{i+1};
                    }} else if(op_a_sig == op_b_sig && op_a > op_b){{ // The second: if the signal is the same, and op_a > op_b
                        //meta.neuron_1_data = (bit<WORDSIZE>) hdr.ann.neuron_id;
                        meta.neuron_2_data = {i};
                        meta.neuron_max_value = hdr.ann.data_{i+1};
                    }}"""

argmax +="""
                }
                reg_neuron_1_data.write(0, meta.neuron_1_data);
                reg_neuron_2_data.write(0, meta.neuron_2_data);
                reg_neuron_max_value.write(0, meta.neuron_max_value);
                // in the argmax function, neuron 3 data and neuron 4 data are not needed, set to 99 just for testing. Could use a different header for this layer so we don't have empty fileds.
"""
for i in range(max_number_of_neurons-2):
  argmax += f"""                meta.neuron_{i+3}_data = 99;
                reg_neuron_{i+3}_data.write(0, meta.neuron_{i+3}_data);
"""
argmax +="            }\n"
print(argmax)



            else if(meta.agg_func == FUNC_ARGMAX){
                // the data to be fowarded (neuron_1_data) is the ID of the switch with highest value.
                // neuron_2_data is the index of the neuron with highest value inside the same switch.
                // the highest data (neuron_max_value) is kept to be compared by other neurons.
                bit<WORDSIZE> op_a = 0;
                bit<WORDSIZE> op_b = 0;
                bit<1> op_a_sig = 0;
                bit<1> op_b_sig = 0;
                if(meta.n_received_stimuli == 1){
                    // if first stimuli, then assume first data received is the higher, then check the remmaining data against it
                    // Neuron 1
                    meta.neuron_1_data = (bit<WORDSIZE>) hdr.ann.neuron_id;
                    meta.neuron_2_data = 0; // neuron_2_data stores the index of the neuron with highest value within same switch
                    meta.neuron_max_value = hdr.ann.data_1;

             

### Identity

In [20]:
ident = "            else if(meta.agg_func == FUNC_IDENTITY){"
for i in range(max_number_of_neurons):
  ident += f"""
                meta.neuron_{i+1}_data = hdr.ann.data_{i+1};
                reg_neuron_{i+1}_data.write(0, meta.neuron_{i+1}_data);\n"""

ident += "            }"
print(ident)

            else if(meta.agg_func == FUNC_IDENTITY){
                meta.neuron_1_data = hdr.ann.data_1;
                reg_neuron_1_data.write(0, meta.neuron_1_data);

                meta.neuron_2_data = hdr.ann.data_2;
                reg_neuron_2_data.write(0, meta.neuron_2_data);

                meta.neuron_3_data = hdr.ann.data_3;
                reg_neuron_3_data.write(0, meta.neuron_3_data);

                meta.neuron_4_data = hdr.ann.data_4;
                reg_neuron_4_data.write(0, meta.neuron_4_data);

                meta.neuron_5_data = hdr.ann.data_5;
                reg_neuron_5_data.write(0, meta.neuron_5_data);

                meta.neuron_6_data = hdr.ann.data_6;
                reg_neuron_6_data.write(0, meta.neuron_6_data);

                meta.neuron_7_data = hdr.ann.data_7;
                reg_neuron_7_data.write(0, meta.neuron_7_data);

                meta.neuron_8_data = hdr.ann.data_8;
                reg_neuron_8_data.write(0, meta.neuron_8_data);

   

### Activation

In [21]:
activation = """

            //  After computing aggregation functions, check if all stimuli have been received to proced to activation function
            tab_n_expected_stimuli.apply();                             // Get the number of expected stimuli for the neuron
            if(meta.n_received_stimuli == meta.n_expected_stimuli){     // Check if the number of expected stimuli was just reached, if yes, the neuron_data is the final value, we should propagate it
                tab_neuron_id.apply();                                  // Get the neuron ID
                if(meta.neuron_id > 0){
                    hdr.ann.neuron_id = meta.neuron_id;                 // Overwrite the fields in the ANN header
                }
                tab_activation_func.apply();                            // Get the neuron activation function
                if(meta.activation_func == FUNC_RELU){"""
for i in range(max_number_of_neurons):
  activation += f"""
                    if(meta.neuron_{i+1}_data & (1 << (WORDSIZE-1)) > 0){{     // Relu: if negative, set data to 0
                        meta.neuron_{i+1}_data = 0;
                    }}
                    hdr.ann.data_{i+1} = meta.neuron_{i+1}_data;                    // Overwrite the fields in the ANN header
"""
activation += """                }
                else if(meta.activation_func == FUNC_IDENTITY){
"""
for i in range(max_number_of_neurons):
  activation += f"                    hdr.ann.data_{i+1} = meta.neuron_{i+1}_data;                    // Overwrite the fields in the ANN header\n"
activation += """                }

                reg_received_stimuli.write(0, 0);                     // Reset the registers related to received stimuli
                reg_n_received_stimuli.write(0, 0);
                ann_forward.apply();                                    // Forward the packet according to the ANN forwarding logic
            }
            else {
                drop();
            }
        }
    }
}
}

/*************************************************************************
****************  E G R E S S   P R O C E S S I N G   *******************
*************************************************************************/

control MyEgress(inout headers hdr,
                inout metadata meta,
                inout standard_metadata_t standard_metadata) {
    apply {}
}

/*************************************************************************
*************   C H E C K S U M    C O M P U T A T I O N   **************
*************************************************************************/

control MyComputeChecksum(inout headers  hdr, inout metadata meta) {
    apply {}
}

/*************************************************************************
***********************  D E P A R S E R  *******************************
*************************************************************************/

control MyDeparser(packet_out packet, in headers hdr) {
    apply {
        packet.emit(hdr.ethernet);
        packet.emit(hdr.ann);
    }
}

/*************************************************************************
***********************  S W I T C H  *******************************
*************************************************************************/

V1Switch(
MyParser(),
MyVerifyChecksum(),
MyIngress(),
MyEgress(),
MyComputeChecksum(),
MyDeparser()
) main;
"""
print(activation)




            //  After computing aggregation functions, check if all stimuli have been received to proced to activation function
            tab_n_expected_stimuli.apply();                             // Get the number of expected stimuli for the neuron
            if(meta.n_received_stimuli == meta.n_expected_stimuli){     // Check if the number of expected stimuli was just reached, if yes, the neuron_data is the final value, we should propagate it
                tab_neuron_id.apply();                                  // Get the neuron ID
                if(meta.neuron_id > 0){
                    hdr.ann.neuron_id = meta.neuron_id;                 // Overwrite the fields in the ANN header
                }
                tab_activation_func.apply();                            // Get the neuron activation function
                if(meta.activation_func == FUNC_RELU){
                    if(meta.neuron_1_data & (1 << (WORDSIZE-1)) > 0){     // Relu: if negative, set data to 0
     

### Full P4 Code


In [22]:
w_sum_str = w_sum_gen(number_of_attributes, hidden_layer_nodes)
w_sum_str += w_sum_gen(hidden_layer_nodes, output_layer_nodes)
code = headers + ingress + apply + normalization + w_sum_str + argmax + ident + activation
print(code)

f = open(p4_file_title, "w")
f.write(code)
f.close()
# save this to file.p4


/* -*- P4_16 -*- */
#include <core.p4>
#include <v1model.p4>

const bit<16> TYPE_IPV4 = 0x800;

#define ET_ANN 0x88B5

#define FUNC_WEIGHTED_SUM_32_TO_32 1
#define FUNC_IDENTITY 2
#define FUNC_RELU 3
#define FUNC_ARGMAX 4
#define FUNC_NORMALIZATION 5
#define FUNC_WEIGHTED_SUM_32_TO_4 6

#define PRECISION 8
#define WORDSIZE 16
#define D_WORDSIZE 32
#define SLACK 8

/*************************************************************************
*********************** H E A D E R S  ***********************************
*************************************************************************/

typedef bit<9>  egressSpec_t;
typedef bit<48> macAddr_t;
typedef bit<32> ip4Addr_t;

header ethernet_t{
    macAddr_t dstAddr;
    macAddr_t srcAddr;
    bit<16>   etherType;
}

header ann_t{
    bit<32> neuron_id;
    bit<WORDSIZE> data_1;
    bit<WORDSIZE> data_2;
    bit<WORDSIZE> data_3;
    bit<WORDSIZE> data_4;
    bit<WORDSIZE> data_5;
    bit<WORDSIZE> data_6;
    bit<WORDSIZE> data_7;
    bit<W

### Topology file generator for topology.json


In [23]:
# #definition of topology
# topology = {
#     'hosts': {
#         'h1': {'ip': '10.0.1.1/24', 'mac': '08:00:00:00:01:11',
#                 'commands':['route add default gw 10.0.1.10 dev eth0',
#                             'arp -i eth0 -s 10.0.1.10 08:00:00:00:01:00']},
#         'h2': {'ip': '10.0.2.2/24', 'mac': '08:00:00:00:02:22',
#                 'commands':['route add default gw 10.0.2.20 dev eth0',
#                             'arp -i eth0 -s 10.0.2.20 08:00:00:00:02:00']}
#     },
#     'switches': {},
#     'links': []
# }

# #populate switches dict
# for i in range(number_of_attributes):
#   switch_name = 's%d'%(i+1)
#   switch_content = { 'runtime_json' : 'topology/s%d'%(i+1) +'-runtime.json' }
#   topology['switches'][switch_name] = switch_content

# for i in range(hidden_layer_nodes):
#   switch_name = 's%d'%(i+51)
#   switch_content = { 'runtime_json' : 'topology/s%d'%(i+51) +'-runtime.json' }
#   topology['switches'][switch_name] = switch_content

# for i in range(output_layer_nodes):
#   switch_name = 's%d'%(i+101)
#   switch_content = { 'runtime_json' : 'topology/s%d'%(i+101) +'-runtime.json' }
#   topology['switches'][switch_name] = switch_content

# topology['switches']['s126'] = { 'runtime_json' : 'topology/s126-runtime.json' }

# #populate links list
# for i in range(number_of_attributes):
#   topology['links'].append(["h1", "s%d"%(i+1)+"-p1"])

# for i in range(number_of_attributes):
#   for j in range(hidden_layer_nodes):
#     topology['links'].append(["s%d"%(i+1)+"-p%d"%(j+51), "s%d"%(j+51)+"-p%d"%(i+1)])

# for i in range(hidden_layer_nodes):
#   for j in range(output_layer_nodes):
#     topology['links'].append(["s%d"%(i+51)+"-p%d"%(j+101), "s%d"%(j+101)+"-p%d"%(i+51)])

# for i in range(output_layer_nodes):
#   topology['links'].append(["s%d"%(i+101)+"-p126", "s126-p%d"%(i+101)])

# topology['links'].append(["s126-p2", "h2"])

# display(topology)

# topology_title = "/content/drive/MyDrive/network-traffic-classification-main/code/topology/topology.json"
# with open(topology_title, "w") as f:
#   json.dump(topology, f, indent = 2)


In [24]:
# expected stimuli
# #hidden
# sum([1<<(i + 1) for i in range(n)])

# #output
# sum([1<<(i + 51) for i in range(n)])

# #expected stimuli
# import sys

# def main(neurons):
#     neurons = [int(x) for x in neurons.split(",")]
#     expected_stimuli = 0
#     for x in neurons:
#         expected_stimuli = expected_stimuli | 1 << x

#     print("dec:", expected_stimuli)
#     print("bin:", bin(expected_stimuli))

# if __name__ == '__main__':
#     main(sys.argv[1])
